In [3]:
import json
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


from scipy import sparse
from fast_pagerank import pagerank
from fast_pagerank import pagerank_power

from itertools import product


pd.set_option('display.max_colwidth', None)

In [17]:
import pandas as pd
test_kps = pd.read_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\page_rank\key_points_test.csv')
test_args = pd.read_pickle(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\page_rank\test_args_with_scores.pkl')

In [18]:
test_args.head(2)

,arg_id,argument,topic,stance,sents,sents_with_scores
0,arg_0_0,Routine child vaccinations isn't mandatory since children don't spread the virus,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations isn't mandatory since children don't spread the virus],"[(Routine child vaccinations isn't mandatory since children don't spread the virus, 0.7141945362091064)]"
1,arg_0_1,Routine child vaccinations should not be mandatory because children may not bear the side effects of it.,Routine child vaccinations should be mandatory,-1,[Routine child vaccinations should not be mandatory because children may not bear the side effects of it.],"[(Routine child vaccinations should not be mandatory because children may not bear the side effects of it., 0.871067464351654)]"


In [19]:
test_topics = test_args.groupby(['topic', 'stance']).agg({
    'sents_with_scores': lambda x: set([item for items in x for item in items]),
    'sents': lambda x: set([item for items in x for item in items])
}).reset_index()

In [20]:
test_topics.head(1)

topic  stance  \
0  Routine child vaccinations should be mandatory      -1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [6]:
test_topics['gt-kps']= test_topics.apply(lambda row: test_kps[(test_kps.topic==row['topic']) & (test_kps.stance==row['stance'])]['key_point'].tolist(), axis=1)

In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [8]:
def gen_match_matrix(model, sents, topic, min_match_score=0):
    sents1 = [topic + ' <SEP> ' + x for x in sents]
    sents1_embeddings = model.encode(sents1)
    sim_matrix = cosine_similarity(sents1_embeddings, sents1_embeddings)
    super_threshold_indices = sim_matrix < min_match_score
    sim_matrix[super_threshold_indices] = 0
    return sim_matrix

In [9]:
def filter_ranked_list(row, model, min_match=0.8):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    filtered_sents = []
    while len(filtered_sents) < 5:
        filtered_sents = []
        for i, s in enumerate(ranked_sents):
            if len(filtered_sents) == 0:
                filtered_sents.append(s)
                print('FFFF ' , s)
            else:
                matching_scores = gen_match_matrix(model, [s]+filtered_sents, row['topic'])
                max_sim = np.max(matching_scores[0][1:])
                #print(matching_scores)
                if max_sim < min_match:
                    print('SSSS ' , s)
                    filtered_sents.append(s)
        
        min_match = min_match + 0.1
        print(min_match)

    return filtered_sents

In [10]:
def apply_page_rank(row, p=0.85, min_quality_score=0.5, min_match_score=0.5, min_len=5, max_len=20):
    cand_sents = [x for x in row['sents_with_scores'] if x[1] > min_quality_score and len(x[0].split()) < max_len and len(x[0].split()) > min_len]
    if len(cand_sents) == 0:
        return []
        print('empty')
    cands, cands_qualities = zip(*cand_sents)
    #print(cands)
    #print(cands_qualities)
    cands_matching_mat = gen_match_matrix(model, cands, row['topic'], min_match_score=min_match_score)
    pr=pagerank(cands_matching_mat, personalize=np.array(cands_qualities), p=p) #it looks like modifying the initial probability doesn't help
    #pr=pagerank(cands_matching_mat, p=p)
    ranked_candidates = list(zip(cands, pr))
    return sorted(ranked_candidates, key=lambda x: -x[1])    

In [23]:
import rouge 

evaluator = rouge.Rouge(metrics=['rouge-l'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

def evalute_topic_by_rouge(row):
    ranked_sents = [x[0] for x in row['ranked_sents']][0:10]
    gt_kps = [x for x in row['gt-kps']]

    r_l = []
    for a,b in product(gt_kps, ranked_sents):
        scores = evaluator.get_scores(a, b)
        r_l.append(scores['rouge-l']['f'])
    
    return np.mean(r_l)

def evalute_topic(row):
    ranked_sents = [x[0] for x in row['ranked_sents']]
    #ranked_sents = [x for x in row['filtered_sents']]
    gt_kps_ranks = []
    for real_kp in row['gt-kps']:
        if real_kp in ranked_sents:
            gt_kps_ranks.append(ranked_sents.index(real_kp)+1)
        else:
            print('kp is not there...')
            gt_kps_ranks.append(len(ranked_sents)+1)
            
    #print(gt_kps_ranks)
    return 1/(sum(gt_kps_ranks)/len(gt_kps_ranks)) if len(gt_kps_ranks) > 0 else 0

In [11]:
test_topics['ranked_sents'] = test_topics.apply(lambda row: apply_page_rank(row, p=0.2, min_quality_score=0.8, min_match_score=0.8), axis=1)

C:\Users\Bilal\anaconda3\envs\cryptocurrencies-kpa\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\anaconda3\envs\cryptocurrencies-kpa\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\anaconda3\envs\cryptocurrencies-kpa\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\anaconda3\envs\cryptocurrencies-kpa\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
C:\Users\Bilal\anaconda3\envs\cr

In [ ]:
test_topics['ranked_kps'] = test_topics.apply(lambda row: filter_ranked_list(row, model, min_match=0.8), axis=1)

In [ ]:
test_topics.head()

In [24]:
test_topics['rouge_scores'] = test_topics.apply(lambda row: evalute_topic_by_rouge(row), axis=1)

In [25]:
test_topics[['topic', 'stance', 'rouge_scores']].head(10)

,topic,stance,rouge_scores
0,Routine child vaccinations should be mandatory,-1,0.230075
1,Routine child vaccinations should be mandatory,1,0.323566
2,Social media platforms should be regulated by the government,-1,0.336829
3,Social media platforms should be regulated by the government,1,0.332772
4,The USA is a good country to live in,-1,0.219639
5,The USA is a good country to live in,1,0.243432


In [13]:
test_topics[['topic', 'stance', 'ranked_kps', 'gt-kps']].to_pickle(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\page_rank\pagerank-test-keypoints.pkl')

In [14]:
keypoints_ds = []
c = 0
N = 4
for i, row in test_topics.iterrows():
    for kp in row['ranked_kps'][0:N]:
        keypoints_ds.append([c, row['topic'], row['stance'], kp])
        c+=1
        
keypoints_df = pd.DataFrame(keypoints_ds, columns=['key_point_id', 'topic', 'stance', 'key_point'])

In [15]:
keypoints_df.head(25)

,key_point_id,topic,stance,key_point
0,0,Routine child vaccinations should be mandatory,-1,Vaccines in children should not be mandatory because they can have consequences for their health in the future
1,1,Routine child vaccinations should be mandatory,-1,"The child population has a low degree of vulnerability, so vaccination is not urgent yet"
2,2,Routine child vaccinations should be mandatory,-1,The vaccine could make your child sick and contain toxic substances.
3,3,Routine child vaccinations should be mandatory,-1,"When vaccines are mandatory, they can infringe on family religious choices"
4,4,Routine child vaccinations should be mandatory,1,child vaccinations should be mandatory to provide decent health care to all.
5,5,Routine child vaccinations should be mandatory,1,"Vaccines save up to 3 million lives a year, protecting children from life-threatening and highly infectious diseases."
6,6,Routine child vaccinations should be mandatory,1,with this we obtain immunity in the immunological system to viral diseases
7,7,Routine child vaccinations should be mandatory,1,"Vaccines are currently available to protect us against at least 20 diseases, including diphtheria"
8,8,Social media platforms should be regulated by the government,-1,the fact that the government regulates social media platforms undermines freedom of expression and thought
9,9,Social media platforms should be regulated by the government,-1,Social media platform should not be regulated by the government because it undermines freedom of speech


In [16]:
keypoints_df.to_csv(r'C:\Users\Bilal\crypto\thesis\my-repos\cryptocurrencies-kpa\data\raw\ArgKP_2021_shared_task\page_rank\pagerank-generated-test-keypoints.csv', index=False)

In [17]:
keypoints_df.topic.value_counts()

Social media platforms should be regulated by the government    25
Routine child vaccinations should be mandatory                  18
The USA is a good country to live in                            18
Name: topic, dtype: int64